In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import glob 
import json
import re
import string
import operator
import collections
import numpy as np
import string
import math
from tqdm import tqdm
from array import array

In [2]:
NONE_STR = '<s>'

In [3]:
def delete_other_letter(line):
    punctuation ="""!"#$%&\()*+,./:;<=>?@[\\]^_`{|}~\\n"""
    re_punctuation ="[{}]+".format(punctuation)
    line =re.sub(re_punctuation, "", line)
    return line.lower()

In [4]:
def read_articles(paths):
    alllines = []
    for path in paths:
        filenames = glob.glob(path+"/*txt")
        for filename in filenames:
            with open(filename, 'r', encoding='utf-8') as fpr:
                data_raw = json.load(fpr)
                article = data_raw['article']
                
                lines = article.split('.')
                for line in lines:
                    line = delete_other_letter(line)
                    alllines.append(line)
                
    return alllines

In [5]:
lines = read_articles(['./RACE/train/high','./RACE/train/middle'])

In [6]:
print(len(lines))

499388


In [7]:
# def word_dict(lines):
#     word_freq = collections.defaultdict(int)
#     for line in lines:
#         for w in line.split(): 
#             word_freq[w] += 1
    
#     return list(word_freq.keys())

In [8]:
# def get_words_matrix(words,lines):                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
#     for line in lines:
#         line_words = line.split()
#         n = len(line_words)
#         for i in range(n-1):
#             ww = line_words[i+1]
#             i = words.index(line_words[i])
#             j = words.index(ww)
#             words_matrix[i][j] += 1

In [9]:
def word_dict(lines,n):
                
#     pre_words= set()
#     be_words = set()
    pre_words = collections.defaultdict(int)
    be_words = collections.defaultdict(int)
    n = n-1
    for line in lines:
        for k in range(n):
            line = NONE_STR+' '+line+' '+NONE_STR
        line_words = line.split()
        for i in range(len(line_words)-(n-1)):
            ngramTemp = ' '.join(line_words[i:i+n])
#             pre_words.add(ngramTemp)
#             be_words.add(line_words[i])
            pre_words[ngramTemp] += 1
            be_words[line_words[i]] += 1
            
#     return list(pre_words.keys()),list(be_words.keys())
    return pre_words,be_words

In [10]:
punctuation_list = ['.',',','?','!','\"','“']

In [11]:
def has_punctuation(word):
    for puc in punctuation_list:
        if word.find(puc) != -1 :
            return True
    return False

In [12]:
def replace_punctuation(word):
    for puc in punctuation_list:
        word = word.replace(puc,'')
    return word

In [13]:
def replace_Chinese_punctuation(content):
    content = content.replace('’','\'')
    content = content.replace('”','')
    return content

In [14]:
def get_options(options_str):
    options_str = replace_Chinese_punctuation(options_str)
    options = []
    opt_rows = options_str.split('#')
    for opt in opt_rows:
        indexA = opt.find('A.')
        indexB = opt.find('B.')
        indexC = opt.find('C.')
        indexD = opt.find('D.')
        As = opt[indexA+2:indexB].strip().lower().split(' ')
        Bs = opt[indexB+2:indexC].strip().lower().split(' ')
        Cs = opt[indexC+2:indexD].strip().lower().split(' ')
        Ds = opt[indexD+2:].strip().lower().split(' ')
        options.append([As,Bs,Cs,Ds])
    return options

In [15]:
def probability(word1,word2):
    index1 = pre_words.index(word1)
    index2 = be_words.index(word2)
#     print(word1,index1,word2,index2)
    pn_word1 = words_matrix[index1].sum()
    pn_word2_under_word1 = words_matrix[index1][index2]
#     print(pn_word1,pn_word2_under_word1)
    prob = float(pn_word2_under_word1)/pn_word1
    if prob == 0:
        prob = 0.0000000000001
    return prob

In [16]:
def read_train_data(n):
    path = './data'
    filenames = glob.glob(path+"/*txt")
    
    answers_list =[]
    options_list = []
    content_list = []
    for filename in filenames:
        with open(filename, 'r', encoding='utf-8') as fpr:
            data_raw = json.load(fpr)
            article = data_raw['article']
            content = get_pre_and_be_words(article,n)
            options_str = data_raw['options']
            options = get_options(options_str)
            answers = list(data_raw['answers'])
            answers_list.append(answers)
            options_list.append(options)
            content_list.append(content)
    return content_list,options_list,answers_list

In [17]:
def auto_select(question,option):
    index = 0
    maxprob = -10000000000
    n = len(option)
    for i in range(n):
#         prob = math.log(probability(question[0],option[i][0]))+math.log(probability(option[i][-1],question[1]))
        prob = probability(question,option[i][0])
        if prob > maxprob:
            maxprob = prob
            index = i
    return chr(ord('A')+index)

In [18]:
def test(content_list,options_list,answers_list,showAll=False,predict=None):
    
    cn = len(content_list)
    right_n = 0
    total_n = 0
    for k in range(cn):
        content = content_list[k]
        answers = answers_list[k]
        options = options_list[k]
    
        n = len(answers)
        total_n += n
        for i in range(n):
            if showAll:
                print('---------'+str(i+1)+'---------')
            if predict is None:
                choice = auto_select(content[i],options[i])
            else:
                choice = predict
            if choice == answers[i]:
                right_n += 1
                if showAll == False:
                    print('---------'+str(i+1)+'---------')
                    print('predict:',choice,'ground_true:',answers[i])
                    print('空的前后词',content[i])
                    print('选项',options[i])
            if showAll:
                print('predict:',choice,'ground_true:',answers[i])
                print('空的前后词',content[i])
                print('选项',options[i])
    accuracy = float(right_n)/total_n
    print('accuracy:',accuracy)

In [19]:
def get_words_matrix(pre_words,be_words,lines,n):  
    n = n-1
    for line in tqdm(lines):
        for k in range(n):
            line = NONE_STR+' '+line+' '+NONE_STR
        line_words = line.split()
        words_n = len(line_words)-n
        for i in range(words_n):
            pre_word = ' '.join(line_words[i:i+n])
                
            be_word = line_words[i+n]
            try:
                i = pre_words.index(pre_word)
                j = be_words.index(be_word)
                words_matrix[i][j] += 1
            except ValueError:
                # 查找不到会报错，找不到就不做处理
                a = None
            
    return words_matrix

In [20]:
def get_pre_and_be_words(article,ngram):
    article = replace_Chinese_punctuation(article)
    art_words = article.split()
    n = len(art_words)
    index = 1
    content = []
    for i in range(n):
        word = art_words[i]
        if str(index) == word:
            pre_word = ''
            if i < ngram-1:
                for k in range(ngram-1-i):
                    pre_word += NONE_STR
            else:
#                 pre_word = art_words[i-1-(ngram-2):i-1]
                pre_word = pre_word.join(art_words[i-1-(ngram-2):i])
            if has_punctuation(pre_word):
                pre_word = ''
                for k in range(ngram-1):
                    pre_word += NONE_STR
            pre_word = replace_punctuation(pre_word)
            content.append(pre_word.lower())
            index += 1
    return content

<font color=blue size=15 face=雅黑>bigram Test</font>

In [21]:
pre_words_dict,be_words_dict = word_dict(lines,2)

In [22]:
pre_words = list(pre_words_dict.keys())
be_words = list(be_words_dict.keys())

In [23]:
m = len(pre_words)
n = len(be_words)
words_matrix = np.zeros((m,n),dtype=np.int32) 
print(words_matrix.shape)

(146579, 146579)


In [24]:
words_matrix = get_words_matrix(pre_words,be_words,lines,2)

100%|██████████| 499388/499388 [24:10<00:00, 344.35it/s]


In [25]:
content_list,options_list,answers_list = read_train_data(2)

In [29]:
print(content_list[0])

["i'd", 'my', 'was', 'to', 'was', 'little', 'in', 'not', 'and', 'was', 'helped', 'something', 'very', 'i', '<s>', 'to', 'ben', '<s>', 'a', 'there']


In [30]:
print(options_list[0])

[[['also'], ['often'], ['even'], ['last']], [['delight'], ['relief'], ['anger'], ['worry']], [['scared'], ['shocked'], ['thrilled'], ['ashamed']], [['talk'], ['stay'], ['meet'], ['settle']], [['bathed'], ['clean'], ['deep'], ['formed']], [['faster'], ['closer'], ['heavier'], ['wiser']], [['trouble'], ['advance'], ['question'], ['battle']], [['real'], ['right'], ['fair'], ['fit']], [['stared'], ['sank'], ['dived'], ['fell']], [['arguing'], ['fighting'], ['shouting'], ['shaking']], [['lead'], ['persuade'], ['carry'], ['keep']], [['happened'], ['occurred'], ['applied'], ['appealed']], [['sharp'], ['pleasant'], ['attractive'], ['familiar']], [['agreed'], ['hesitated'], ['doubted'], ['knew']], [['fortunately'], ['frankly'], ['sadly'], ['suddenly']], [['return'], ['relax'], ['speak'], ['leave']], [['joked'], ['turned'], ['listened'], ['pointed']], [['created'], ['honored'], ['saved'], ['guided']], [['coincidence'], ['change'], ['pity'], ['pain']], [['on', 'board'], ['in', 'time'], ['for', 's

In [31]:
print(answers_list[0])

['D', 'A', 'C', 'C', 'A', 'B', 'A', 'B', 'C', 'D', 'C', 'B', 'D', 'D', 'A', 'D', 'B', 'C', 'A', 'B']


<font color=black size=5 face=雅黑>全选A/B/C/D 观察正确率</font>

In [35]:
test(content_list,options_list,answers_list,showAll=False,predict='A')

---------2---------
predict: A ground_true: A
空的前后词 my
选项 [['delight'], ['relief'], ['anger'], ['worry']]
---------5---------
predict: A ground_true: A
空的前后词 was
选项 [['bathed'], ['clean'], ['deep'], ['formed']]
---------7---------
predict: A ground_true: A
空的前后词 in
选项 [['trouble'], ['advance'], ['question'], ['battle']]
---------15---------
predict: A ground_true: A
空的前后词 <s>
选项 [['fortunately'], ['frankly'], ['sadly'], ['suddenly']]
---------19---------
predict: A ground_true: A
空的前后词 a
选项 [['coincidence'], ['change'], ['pity'], ['pain']]
---------5---------
predict: A ground_true: A
空的前后词 could
选项 [['see'], ['find'], ['guess'], ['learn']]
---------8---------
predict: A ground_true: A
空的前后词 i
选项 [['chatted', 'with'], ['looked', 'at'], ['laughed', 'at'], ['talked', 'about']]
---------12---------
predict: A ground_true: A
空的前后词 and
选项 [['left'], ['acted'], ['sat', 'down'], ['calmed', 'down']]
---------13---------
predict: A ground_true: A
空的前后词 had
选项 [['wanted'], ['tried'], ['ordered']

In [36]:
test(content_list,options_list,answers_list,showAll=False,predict='B')

---------6---------
predict: B ground_true: B
空的前后词 little
选项 [['faster'], ['closer'], ['heavier'], ['wiser']]
---------8---------
predict: B ground_true: B
空的前后词 not
选项 [['real'], ['right'], ['fair'], ['fit']]
---------12---------
predict: B ground_true: B
空的前后词 something
选项 [['happened'], ['occurred'], ['applied'], ['appealed']]
---------17---------
predict: B ground_true: B
空的前后词 ben
选项 [['joked'], ['turned'], ['listened'], ['pointed']]
---------20---------
predict: B ground_true: B
空的前后词 there
选项 [['on', 'board'], ['in', 'time'], ['for', 'sure'], ['on', 'purpose']]
---------3---------
predict: B ground_true: B
空的前后词 newspaper
选项 [['flat'], ['open'], ['cut'], ['fixed']]
---------7---------
predict: B ground_true: B
空的前后词 the
选项 [['direct'], ['familiar'], ['strange'], ['funny']]
---------9---------
predict: B ground_true: B
空的前后词 as
选项 [['the', 'waiter'], ['time'], ['i'], ['the', 'dinner']]
---------16---------
predict: B ground_true: B
空的前后词 my
选项 [['care'], ['surprise'], ['worry'],

In [37]:
test(content_list,options_list,answers_list,showAll=False,predict='C')

---------3---------
predict: C ground_true: C
空的前后词 was
选项 [['scared'], ['shocked'], ['thrilled'], ['ashamed']]
---------4---------
predict: C ground_true: C
空的前后词 to
选项 [['talk'], ['stay'], ['meet'], ['settle']]
---------9---------
predict: C ground_true: C
空的前后词 and
选项 [['stared'], ['sank'], ['dived'], ['fell']]
---------11---------
predict: C ground_true: C
空的前后词 helped
选项 [['lead'], ['persuade'], ['carry'], ['keep']]
---------18---------
predict: C ground_true: C
空的前后词 <s>
选项 [['created'], ['honored'], ['saved'], ['guided']]
---------1---------
predict: C ground_true: C
空的前后词 i
选项 [['knew'], ['understood'], ['noticed'], ['recognized']]
---------4---------
predict: C ground_true: C
空的前后词 was
选项 [['hoping'], ['thinking'], ['pretending'], ['continuing']]
---------10---------
predict: C ground_true: C
空的前后词 became
选项 [['true'], ['hopeful'], ['clear'], ['possible']]
---------14---------
predict: C ground_true: C
空的前后词 he
选项 [['met'], ['caught'], ['followed'], ['discovered']]
---------20

In [38]:
test(content_list,options_list,answers_list,showAll=False,predict='D')

---------1---------
predict: D ground_true: D
空的前后词 i'd
选项 [['also'], ['often'], ['even'], ['last']]
---------10---------
predict: D ground_true: D
空的前后词 was
选项 [['arguing'], ['fighting'], ['shouting'], ['shaking']]
---------13---------
predict: D ground_true: D
空的前后词 very
选项 [['sharp'], ['pleasant'], ['attractive'], ['familiar']]
---------14---------
predict: D ground_true: D
空的前后词 i
选项 [['agreed'], ['hesitated'], ['doubted'], ['knew']]
---------16---------
predict: D ground_true: D
空的前后词 to
选项 [['return'], ['relax'], ['speak'], ['leave']]
---------2---------
predict: D ground_true: D
空的前后词 <s>
选项 [['since'], ['even', 'if'], ['though'], ['as', 'if']]
---------6---------
predict: D ground_true: D
空的前后词 my
选项 [['menu'], ['bill'], ['paper'], ['food']]
---------11---------
predict: D ground_true: D
空的前后词 the
选项 [['restaurant'], ['washroom'], ['office'], ['kitchen']]
---------15---------
predict: D ground_true: D
空的前后词 he
选项 [['was', 'to', 'beat'], ['was', 'dealing', 'with'], ['was', 'to',

<font color=black size=5 face=雅黑>采用bigram模型后，正确率</font>

In [40]:
test(content_list,options_list,answers_list,True)

---------1---------
predict: A ground_true: D
空的前后词 i'd
选项 [['also'], ['often'], ['even'], ['last']]
---------2---------
predict: C ground_true: A
空的前后词 my
选项 [['delight'], ['relief'], ['anger'], ['worry']]
---------3---------
predict: B ground_true: C
空的前后词 was
选项 [['scared'], ['shocked'], ['thrilled'], ['ashamed']]
---------4---------
predict: B ground_true: C
空的前后词 to
选项 [['talk'], ['stay'], ['meet'], ['settle']]
---------5---------
predict: D ground_true: A
空的前后词 was
选项 [['bathed'], ['clean'], ['deep'], ['formed']]
---------6---------
predict: A ground_true: B
空的前后词 little
选项 [['faster'], ['closer'], ['heavier'], ['wiser']]
---------7---------
predict: B ground_true: A
空的前后词 in
选项 [['trouble'], ['advance'], ['question'], ['battle']]
---------8---------
predict: B ground_true: B
空的前后词 not
选项 [['real'], ['right'], ['fair'], ['fit']]
---------9---------
predict: D ground_true: C
空的前后词 and
选项 [['stared'], ['sank'], ['dived'], ['fell']]
---------10---------
predict: D ground_true: D
空的前

In [41]:
test(content_list,options_list,answers_list,False)

---------8---------
predict: B ground_true: B
空的前后词 not
选项 [['real'], ['right'], ['fair'], ['fit']]
---------10---------
predict: D ground_true: D
空的前后词 was
选项 [['arguing'], ['fighting'], ['shouting'], ['shaking']]
---------14---------
predict: D ground_true: D
空的前后词 i
选项 [['agreed'], ['hesitated'], ['doubted'], ['knew']]
---------16---------
predict: D ground_true: D
空的前后词 to
选项 [['return'], ['relax'], ['speak'], ['leave']]
---------2---------
predict: D ground_true: D
空的前后词 <s>
选项 [['since'], ['even', 'if'], ['though'], ['as', 'if']]
---------5---------
predict: A ground_true: A
空的前后词 could
选项 [['see'], ['find'], ['guess'], ['learn']]
---------10---------
predict: C ground_true: C
空的前后词 became
选项 [['true'], ['hopeful'], ['clear'], ['possible']]
---------11---------
predict: D ground_true: D
空的前后词 the
选项 [['restaurant'], ['washroom'], ['office'], ['kitchen']]
---------12---------
predict: A ground_true: A
空的前后词 and
选项 [['left'], ['acted'], ['sat', 'down'], ['calmed', 'down']]
--------

<font color=blue size=15 face=雅黑>trigram Test</font>

In [26]:
N = 3
pre_words_dict,be_words_dict = word_dict(lines,N)
be_words = list(be_words_dict.keys())

In [27]:
print('未处理过得pre_words length:',len(pre_words_dict.keys()))

未处理过得pre_words length: 1467004


In [28]:
#对pre_words做特殊处理，因为3-gram时，pre_words过大
pre_words = []
for key, value in pre_words_dict.items():
    if value > 4:
        pre_words.append(key)

In [29]:
print('处理过得pre_words length:',len(pre_words))

处理过得pre_words length: 167397


In [30]:
m = len(pre_words)
n = len(be_words)
words_matrix = np.zeros((m,n),dtype=np.int32) 
print(words_matrix.shape)

(167397, 146579)


处理过的pre_words大小缩小了将近10倍，但是也带来一个问题，就是一些词的组合可能查找不到了

In [ ]:
words_matrix = get_words_matrix(pre_words,be_words,lines,N)

  5%|▍         | 22880/499388 [15:21<4:43:01, 28.06it/s] 

In [ ]:
content_list,options_list,answers_list = read_train_data(N)

In [ ]:
test(content_list,options_list,answers_list,True)

In [ ]:
test(content_list,options_list,answers_list,False)